%%html
<style>table {float:left}</style>

In [1]:
import sys
print('python version:', sys.version_info)
import sqlalchemy
print('sqlalchemy version:',sqlalchemy.__version__)

python version: sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
sqlalchemy version: 1.3.13


 # Pythonとデータベース
<br>
Pythonから利用できるデータベースは、 MySQL, PostgreSQL, SQLite, Oracleなど多岐にわたります。<br>
PythonのデータベースAPI仕様は、PEP 249<sup>※1</sup>で定義されており、それぞれのデータベースに対応して実装<sup>※2</sup>が存在します。
データベースAPIを使用すればデータベースプログラミングが可能ですが、SQLを直接記述する必要があります。<br><br>
SQLを直接記述するプログラムは規模が大きくなると複雑になる傾向があります。そのため最近のデータベース開発では
オブジェクトとデータベースのレコードを関連付ける仕組みである「O/Rマッパー」を利用してデータベース開発を行うケースが増えています。<br><br>

<sup>※1</sup> https://www.python.org/dev/peps/pep-0249/#exceptions <br>
<sup>※2</sup> https://docs.python.org/ja/3/library/sqlite3.html<br><br>

# SQLAlchemy
<br>
SQLAlchemyは、データベースやSQLに関連する機能を提供するライブラリです。テーブル関連の処理やSQLを扱うための便利なモジュールを提供しています。
「O/Rマッパー（ Object / Relational Mapper ）」は提供される機能の１つであり、「O/Rマッパー」を使用せずに開発することも可能です。SQLAlchemyではこのDataMapperパターンを使用してテーブルとオブジェクトのマッピングを行い、開発を進めます。<br><br>

SQLAlchmeyは非常に高機能であり、SQLite,MySQL,PostgerSQLなどのオープンソースのデータベースや、OracleやMicrosoft SQL Serverなどの商用データベースもサポートしています。<br><br>

# データベースへの接続
<br>
データベースの接続には、<u>create_engine()</u>関数を使用します。<ul><li>第１引数・・・「接続するデータベース」を指定します。<br>今回は、メモリーベースである SQLite3 を指定しています。<br><br>
<li>第2引数・・・echo= はログ出力の有効化フラグです。<br>Trueを指定するとコンソールにSQLAlchemyが発行したSQLなどが出力されます。<br><br>

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

# テーブルの作成
SQLAlchemyを使用してテーブルを作成します。<br>テーブルは次の表のように書籍の情報を格納するbooksテーブルを作成します。<br><br>SQLAlchemyでは、テーブルを表すオブジェクトとマッピングされるオブジェクトを定義し、それらを関連付けます。
定義と関連付けはdeclarativeを使って同時に行っています。<br><br>
#### ◇ booksテーブルの定義
|<div align='left'>名前</div>|<div align='left'>カラム名</div>|<div align='left'>データタイプ</div>
|-----|-----|-----
|<div align='left'>ID</div>|<div align='left'>id</div>|<div align='left'>INTEGER</div>
|<div align='left'>タイトル</div>|<div align='left'>title</div>|<div align='left'>VARCHAR(255)</div>
|<div align='left'>価格</div>|<div align='left'>price</div>|<div align='left'>INTEGER</div>
|<div align='left'>メモ</div>|<div align='left'>memo</div>|<div align='left'>TEXT</div>

### Bookクラス（オブジェクト）定義
<br><hr><br>
<u>Bookクラス</u>の定義は次のように行います。</u><br><br>
<ul>
1. declarative をインポートします。<br><br>
2. declarative.declarative_base() 関数で、Baseクラス を生成します。<br><br>
3. フィールド定義に使用する Column, Integer, Unicode, UnicodeText をインポートします。<br><br>
4. Baseクラスを継承した Bookクラス を生成します。<br>
Bookクラスでは、データベースのカラムに対応するフィールドの定義を行います。<br><br>
5. \_\_tablename\_\_ = 'books' でテーブル名を指定します。<br><br>
6. Columnクラスを使ってフィールドを定義します。<br><br></ul>
<hr><br>
Columnクラスのコンストラクタの [第１引数] にはフィールドの型、[第２引数] 以降にはオプションを指定します。

<ul><li>primary_key=True と指定すると、そのカラムはプライマリーキーとなります。
    <li>nullable=False と指定すると、nullが 許可さなくなります。
    <li>テーブルにデータが保存されるときのでフィルと値は default='<デフォルト値>' で指定します。</ul>
<br>
declarative を使ってクラス宣言したので、booksテーブル とマッピングされた Bookオブジェクト がプログラム内で使用できるようになります。
<br><br>

In [ ]:
from sqlalchemy.ext import declarative
Base = declarative.declarative_base()
from sqlalchemy import Column, Integer, Unicode, UnicodeText
class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(Unicode(100), nullable=False)
    price = Column(Integer, nullable=False)
    memo = Column(UnicodeText)
    def __repr_(self):
        return "<Book('%s', '%s', '%s')>" % (self.title, self.price, self.memo)

### テーブル作成
<br>
テーブルは、Base.metadat の create_all()メソッド にcreate_engine() で作成したengineを渡して作成します。
Metadata はテーブルなどの情報を登録するためのオブジェクトです。<br>create_all() メソッドを呼び出すとデータベース上にテーブルが作成されます。<br><br>

In [ ]:
Base.metadata.create_all(engine)

# テーブルの操作
<br>
テーブルとオブジェクトのマッピングが完了すると、データベースにアクセスして基本的なCRUD操作ができるようになります。<br><br>
CRUDとは、<ul>__C__reate (登録)、__R__ead（読み取り）、__U__pdate (更新）、__D__elete（削除）</ul>の略称です。<br><br>
    
### セッションの作成
SQLAlchemyではセッションを通してオブジェクトの取得や更新を行います。まずは次に示すようにSessionインスタンスを作成します。
1. sqlchemey.ormのsessionmakerをインポートします。
2. sessionmakerでengineを関連付け(bind)してSessionクラスを作成します。
3. Sessionクラスのコンストラクターを呼び出してインスタンスsessionを生成します。
<br><br>

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

### トランザクション
SQLAlchemyでオブジェクトの登録、更新、削除を行った後…<br><br>
<ul><li>session.commit()メソッドを呼び出して、<br>オブジェクトの更新をデータベースに反映します。<br><br>
<li>（何らかのエラーが発生して）トランザクションをロールバックしたい場合には、<br>session.rollback()メソッドを呼び出します。</ul><br>


### C. オブジェクトの登録<sub>（CREATE / CRUD）</sub>
セッションを生成したら、Bookクラスのオブジェクトをデータベースに登録します。<br>
<ul><li>オブジェクトを登録するにはBookクラスのインスタンスを生成し、<span style="color:orange">__session.add()__</span>メソッドの第１引数に渡します。<br>
<li>Integer型のカラムに、primary_key=Trueが指定されていると、そのカラムは自動的に番号が付けられます。</ul>

In [ ]:
book = Book(title= 'やさしいPython', price='2580', memo = 'SBクリエイティブ')
session.add(book)
session.commit()

<span style="color:orange">__add_all()__メソッド</span>を使用すると<br>一度に複数のBookインスタンスを渡してデータベースに登録出来ます。

In [ ]:
session.add_all([
    Book(title= 'スッキリわかるJava入門', price='2680', memo = 'インプレス'),
    Book(title= '独習PHP', price='3520', memo = '翔泳社'),
])
session.commit()

### R. オブジェクトの取得<sub>（READ / CRUD）</sub>
オブジェクトの登録ができたら、取得を行います。<br>オブジェクトの取得は<span style="color:orange">session.query()</span>を実行してQueryオブジェクトを生成します。<br>
QueryオブジェクトはSQLを実行するためのオブジェクトです。

In [ ]:
query = session.query(Book)

<ul><span style="color:orange">【 all( ) メソッド】</span> 全てのオブジェクトを取得できます。</ul>

In [ ]:
for book in query.all():
    print (book.title)
    

<ul><span style="color:orange">【 get( ) メソッド】 </span>プライマリーキーを用いてオブジェクトを取得できます。<ul>

In [ ]:
book = query.get(1)
print( book.title)

<ul><span style="color:orange">【 filter( ) メソッド】</span> 条件を付けてオブジェクトを取得できます。</ul>

In [ ]:
book = query.filter(Book.title == 'スッキリわかるJava入門').first()
print(book.title)

<ul> (__filter( )__メソッドをつなげて、「複数の条件」を指定することができます。)</ul>

In [ ]:
book = query.filter(Book.title == '独習PHP').filter(Book.id == 3).first()
print(book.title)

<ul><span style="color:orange">【 count( ) メソッド】</span> テーブル上のレコード数を取得できます。</ul>

In [ ]:
count = query.count()
print(count)

### U. オブジェクトの更新<sub>（UPDATE / CRUD）</sub>
オブジェクトの更新は、<ol><li>〇〇.price=2500__ のようにオブジェクトの値を変更して、<li><span style="color:orange">__session.commit( )__</span> を呼び出します。</ol>

In [ ]:
book = query.get(1)
book.price = 2500
session.commit()
book = query.get(1)
print(book.price)

### D. オブジェクトの削除<sub>（DELETE / CRUD）</sub>
オブジェクトの削除は、<ol><li><span style="color:orange">__session.delete( )__</span>にBookインスタンスを渡してから、<li><span style="color:orange">__session.commit( )__</span>を呼び出します。</ol>

In [ ]:
book = query.get(1)
session.delete(book)
session.commit()
book = query.get(1)
print(book)

for book in query.all():
    print(book.title)